In [6]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
drive.mount('/content/drive')

%cd /content/drive/MyDrive/NER

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NER


In [7]:
!ls /content/drive/MyDrive/NER



'1. LM Data Generation.ipynb'	       '5 Final Entitiy Assign..ipynb'
'2.1 LM Pretrained Model'	       'Copy of 2. Language Modelling.ipynb'
'2. Language Modelling.ipynb'	        Final.csv
'3.1 Trained NER Model- 280-10 epoch'   predictions.pkl
'3. SCIBERT_TRAIN.ipynb'	        pre_train_text.txt
'4. SCIBERT_INFER.ipynb'	        ScopusJoournalsAll.xlsx


In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 32.5 MB/s 
     |████████████████████████████████| 6.6 MB 47.4 MB/s 
     |████████████████████████████████| 596 kB 68.3 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
from transformers import AutoModelWithLMHead, AutoTokenizer, AdamW, get_linear_schedule_with_warmup 
from transformers import Trainer, TrainingArguments
from transformers import  DataCollatorForLanguageModeling, LineByLineTextDataset
from torch.utils.data import DataLoader, Dataset


In [9]:
class config:
    MODEL_PATH = "allenai/scibert_scivocab_uncased"
    TOKENIZER_PATH = "allenai/scibert_scivocab_uncased"
    SAVE_DIR = "./"
    BATCH_SIZE = 32
    EPOCHS = 25
    TOTAL_LEN  = 256
    SAVE_STEPS = 10000

In [10]:
model = AutoModelWithLMHead.from_pretrained(config.MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_PATH)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

dataset= LineByLineTextDataset(
    tokenizer = tokenizer,  
    file_path = 'pre_train_text.txt',
    block_size = config.TOTAL_LEN  
)
print('No. of lines: ', len(dataset))



/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCl

No. of lines:  52679


In [ ]:
training_args = TrainingArguments(
    output_dir=config.SAVE_DIR,
    overwrite_output_dir=True,
    num_train_epochs=config.EPOCHS,
    per_device_train_batch_size=config.BATCH_SIZE,
    save_steps=config.SAVE_STEPS,
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,

)
trainer.train()
trainer.save_model(config.SAVE_DIR)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 52679
  Num Epochs = 25
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 41175


Step,Training Loss
500,1.438500
1000,1.421800
1500,1.406100
2000,1.386600
2500,1.374400
3000,1.374600
3500,1.357100
